### Download dos relatórios em formato PDF, agrupados por safra.


In [1]:
import os
import re
import csv
import requests
import logging
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import gc
from typing import List, Tuple
from tqdm import tqdm

# Configura log para terminal e arquivo
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler()]
)

def validar_safra(safra: str) -> None:
    if not re.fullmatch(r'\d{6}', safra):
        raise ValueError(f"Safra '{safra}' deve estar no formato YYYYMM.")
    try:
        datetime.strptime(safra, '%Y%m')
    except ValueError:
        raise ValueError(f"Safra '{safra}' não representa uma data válida.")

def gerar_lista_safras(safra_inicio: str, safra_fim: str) -> List[str]:
    validar_safra(safra_inicio)
    validar_safra(safra_fim)
    data_inicio = datetime.strptime(safra_inicio, '%Y%m')
    data_fim = datetime.strptime(safra_fim, '%Y%m')
    if data_inicio > data_fim:
        raise ValueError("A safra inicial deve ser anterior ou igual à safra final.")

    lista = []
    safra_atual = data_inicio
    while safra_atual <= data_fim:
        lista.append(safra_atual.strftime('%Y%m'))
        ano = safra_atual.year + (safra_atual.month // 12)
        mes = (safra_atual.month % 12) + 1
        safra_atual = datetime(ano, mes, 1)
    return lista

def baixar_pdf(codigo: int, safra: str, url_base: str, pasta_safra: str) -> Tuple[str, int, str, str, str]:
    nome_arquivo_pdf = os.path.join(pasta_safra, f"unidade_{codigo}.pdf")
    if os.path.exists(nome_arquivo_pdf):
        datahora = datetime.fromtimestamp(os.path.getmtime(nome_arquivo_pdf)).strftime("%Y-%m-%d %H:%M:%S")
        return (safra, codigo, "Ignorado", "Arquivo existente", datahora)

    url = f"{url_base}{codigo}&format=pdf"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        with open(nome_arquivo_pdf, 'wb') as f:
            f.write(response.content)
        datahora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        return (safra, codigo, "Sucesso", "Download OK", datahora)
    except requests.RequestException as e:
        datahora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        return (safra, codigo, "Erro", str(e), datahora)

def download_pdfs(safra_inicio: str, safra_fim: str, unidade_inicio: int, unidade_fim: int, diretorio: str, max_threads: int = 5) -> List[Tuple[str, int, str, str, str]]:
    inicio_tempo = time.time()
    lista_safras = gerar_lista_safras(safra_inicio, safra_fim)
    codigos = list(range(unidade_inicio, unidade_fim + 1))
    resultados = []
    total_tarefas = len(lista_safras) * len(codigos)

    logging.info(f"Iniciando download de {total_tarefas} arquivos...")

    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        for safra in lista_safras:
            pasta_safra = os.path.join(diretorio, safra)
            os.makedirs(pasta_safra, exist_ok=True)
            url_base = (
                f"https://www.tjsp.jus.br/APP/ProdutividadePrimeiraInstancia/Report/RelatorioCorreicao"
                f"?anoMesInicial={safra}01&anoMesFinal={safra}01&codigoUnidade="
            )

            logging.info(f"Processando safra {safra}...")
            futuros_safra = [
                executor.submit(baixar_pdf, codigo, safra, url_base, pasta_safra) for codigo in codigos
            ]

            for future in tqdm(as_completed(futuros_safra), total=len(futuros_safra), desc=f"Safra {safra}"):
                resultados.append(future.result())

            gc.collect()

    caminho_csv = os.path.join(diretorio, "relatorio_downloads.csv")
    modo_csv = 'a' if os.path.exists(caminho_csv) else 'w'
    with open(caminho_csv, modo_csv, newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        if modo_csv == 'w':
            writer.writerow(["safra", "codigo_unidade", "status", "mensagem", "data_download"])
        writer.writerows(resultados)

    fim_tempo = time.time()
    duracao_total = fim_tempo - inicio_tempo
    logging.info(f"Download finalizado em {duracao_total:.1f} segundos.")

    sucesso = sum(1 for r in resultados if r[2] == "Sucesso")
    ignorado = sum(1 for r in resultados if r[2] == "Ignorado")
    erro = sum(1 for r in resultados if r[2] == "Erro")

    logging.info(f"\nTotal de arquivos com sucesso: {sucesso:,}")
    logging.info(f"\nTotal de arquivos ignorados: {ignorado:,}")
    logging.info(f"\nTotal de erros: {erro:,}")


In [2]:
download_pdfs(
    safra_inicio="202401",
    safra_fim="202507",
    unidade_inicio=1,
    unidade_fim=3,
    diretorio="data-raw",
    max_threads=5
)

2025-07-15 18:29:20,013 [INFO] Iniciando download de 57 arquivos...
2025-07-15 18:29:20,018 [INFO] Processando safra 202401...
Safra 202401: 100%|██████████| 3/3 [00:00<?, ?it/s]
2025-07-15 18:29:20,115 [INFO] Processando safra 202402...
Safra 202402: 100%|██████████| 3/3 [00:02<00:00,  1.49it/s]
2025-07-15 18:29:22,185 [INFO] Processando safra 202403...
Safra 202403: 100%|██████████| 3/3 [00:00<?, ?it/s]
2025-07-15 18:29:22,266 [INFO] Processando safra 202404...
Safra 202404: 100%|██████████| 3/3 [00:00<?, ?it/s]
2025-07-15 18:29:22,331 [INFO] Processando safra 202405...
Safra 202405: 100%|██████████| 3/3 [00:00<?, ?it/s]
2025-07-15 18:29:22,387 [INFO] Processando safra 202406...
Safra 202406: 100%|██████████| 3/3 [00:00<?, ?it/s]
2025-07-15 18:29:22,449 [INFO] Processando safra 202407...
Safra 202407: 100%|██████████| 3/3 [00:00<?, ?it/s]
2025-07-15 18:29:22,515 [INFO] Processando safra 202408...
Safra 202408: 100%|██████████| 3/3 [00:00<?, ?it/s]
2025-07-15 18:29:22,587 [INFO] Proce